In [1]:
from main_library import *

################################
## data load / adjustements*
extractDate = date_load()

### pour l'instant ne fonctionne pas
## demander à Eric de relancer la machine sur sandbox
# get_call_info()

# get_call_info_europa()

proj = projects_load()
proj_id_signed = proj.project_id.unique()

prop = proposals_load()
proj = proj_add_cols(prop, proj)

stage_p =  ['REJECTED' ,'NO_MONEY' ,'MAIN', 'RESERVE', 'INELIGIBLE', 'WITHDRAWN', 'INADMISSIBLE', None]
prop1 = proposals_status(prop, proj_id_signed, stage_p)  
# np.save("data_files/applicants_columns.npy", prop_cols)

###########################################
# proposals fix
# projects missing from proposals
call_to_integrate, call_miss = proposals_id_missing(prop1, proj, extractDate)

# project data missing in proposals if call already in proposals then add this
proj1 = proj_id_miss_fixed(prop1, proj, call_to_integrate)
call_miss = list(set(call_miss)-set(call_to_integrate))
proj = proj.loc[~proj.callId.isin(call_miss)]

# merge proj + prop
print('### MERGED PROPOSALS/PROJECTS')
if len(proj1)==0:
    prop2=pd.concat([proj,prop1], ignore_index= True)
else:
    prop2 = pd.concat([prop1, proj1, proj], ignore_index = True)

prop2 = prop2.loc[~((prop2.status_code=='REJECTED')&(prop2.stage=='successful'))]
print(f"- result - merged all: {len(prop2)},\n{prop2[['stage','status_code']].value_counts()}")

merged = copy.deepcopy(prop2)
merged = dates_year(merged)
merged = strings_v(merged)
merged = url_to_clean(merged)
merged.mask(merged=='', inplace=True)
merged = empty_str_to_none(merged)      
merged.rename(columns={
    'freekw':'free_keywords',
    'callDeadlineDate':'call_deadline', 
    'callId':'call_id', 
    'submissionDate':'submission_date',
    'startDate':'start_date',
    'endDate':'end_date', 
    'ecSignatureDate':'signature_date'}, inplace=True)

if any(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1]):
    print(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1])

merged = merged_panels(merged)
merged = merged_topics(merged)
merged = merged_actions(merged)

# calls list
calls = call(PATH_SOURCE+FRAMEWORK+'/')

print("\n### CALLS+MERGED")
if len(merged.loc[merged.call_id.isnull()])>0:
        print(f"1 - ATTENTION : manque des call_id: {merged.loc[merged.call_id.isnull(), 'project_id']}")
else:
    call_id = merged[['call_id', 'call_deadline']].drop_duplicates()
    print(f"2 - CALL_ID de merged -> nb call+deadline: {len(call_id)}, nb call unique: {call_id.call_id.nunique()} ")

calls = calls_to_check(calls, call_id)

projects = projects_complete_cleaned(merged, extractDate)

#############################################################
##### PARTICIPATIONS
part = participants_load(proj)
# conserve uniquement les projets présents dans proposals et applicants
part = part.loc[part.project_id.isin(projects.project_id.unique())]
print(f"- size part hors proj manquant: {len(part)}")
part = part_role_type(part)
part = erc_role(part, projects)

#### APPLICANTS
app = applicants_load(prop)
# conserve uniquement les projets présents dans proposals et applicants
app1 = app.loc[app.project_id.isin(projects.project_id.unique())] 
print(f"- size app1 hors proj exclus: {len(app1)}")

app_missing_pid = projects.loc[(projects.stage=='evaluated')&(~projects.project_id.isin(app1.project_id.unique())), 'project_id'].unique()
tmp = part[part.project_id.isin(app_missing_pid)]
app1 = part_miss_app(tmp, app1)

#redressement accelerator
acc_folio = pd.read_csv(f"{PATH_SOURCE}{FRAMEWORK}/eic_fund_portfolio.csv", sep=';', dtype={'PROPOSAL_NBR':str})
print(f"size acc_folio: {len(acc_folio)}")
acc = (app1.loc[(app1.project_id.isin(acc_folio.PROPOSAL_NBR.unique()))&(app1.role=='Coordinator'),['project_id', 'role']]
       .merge(acc_folio[['PROPOSAL_NBR','GRANT_REQUESTED']], how='inner', left_on='project_id', right_on='PROPOSAL_NBR')
       .drop(columns='PROPOSAL_NBR'))
print(f"size acc: {len(acc)}")
app1 = app1.merge(acc, how='left', on=['project_id', 'role'])
app1.loc[app1.requestedGrant.isnull(), 'requestedGrant'] = app1.GRANT_REQUESTED
app1.drop(columns=['GRANT_REQUESTED'], inplace=True)

app1 = app_role_type(app1)
app1 = erc_role(app1, projects)

del app

####
# verification Etat des participations
part = check_multiP_by_proj(part)
app1 = check_multiA_by_proj(app1)


### STEP2
lien = merged_partApp(app1, part)
lien = nuts_lien(app1, part, lien)

# ENTITIES
entities = entities_load(lien)

list_codeCountry = list(set(entities.countryCode.to_list()+lien.countryCode.to_list()))
countries, countryCode_err = country_load(FRAMEWORK, list_codeCountry)
if countryCode_err:
    ccode=json.load(open("data_files/countryCode_match.json"))
    for i in ccode:
        for k,v in i.items():
            lien.loc[lien.countryCode==k, 'countryCode'] = v
            entities.loc[entities.countryCode==k, 'countryCode'] = v
cc_code = (countries[['countryCode', 'country_code_mapping']]
           .rename(columns={'countryCode':'iso2', 'country_code_mapping':'iso3'})
           .drop_duplicates())

lien = lien.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2')
lien = lien.merge(cc_code, how='left', left_on='proposal_countryCode', right_on='iso2').drop(columns='iso2')
lien = (lien.rename(columns={'iso3_x':'country_code_mapping', 'iso3_y':'proposal_country_code_mapping'})
        .drop(columns=['countryCode', 'proposal_countryCode']))
lien.to_pickle(f"{PATH_CLEAN}lien.pkl")

entities = (entities.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2')
            .rename(columns={'iso3':'country_code_mapping'}))

part = part.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2')
app1 = app1.merge(cc_code, how='left', left_on='countryCode', right_on='iso2').drop(columns='iso2')

#ENTITIES +LIEN
entities = entities_cleaning(entities)

### LAST DATE of EXTRACTED DATA
[{'extraction_date': '2024-10-09', 'framework': 'HORIZON'}]

### LOADING PROJECTS data
- no new columns
- result -> dowloaded projects:17382, retained projects:17382, pb:0
- liste des colonnes conservées:
Index(['project_id', 'callId', 'callDeadlineDate', 'typeOfActionCode',
       'acronym', 'status_code', 'startDate', 'endDate', 'ecSignatureDate',
       'title', 'abstract', 'url', 'topicCode', 'duration', 'total_cost',
       'eu_reqrec_grant', 'number_involved', 'framework', 'lastUpdateDate',
       'totalGrant', 'nationalContribution', 'otherContribution', 'freekw',
       'stage'],
      dtype='object')

### LOADING PROPOSALS data
- no new columns
1- empty cols -> Attention ! vérifier les variables manquantes->[]
- result -> dowloaded proposals:92864, retained proposals:92864, pb:0

### ADD COLS TO PROJECTS FROM PROPOSALS
- liste des variables PROJ en plus: ['startDate', 'otherContribution', 'ecSignatureDate', 'status_code', 'nationalContribution',

c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


- code panel existant pour d'autres programmes que erc/msca: données exportées

- size merged after add panels: 102295
### TOPICS
1 - topics -> 2448
2 - divisions -> 3172
erc : destination_code à null après traitement
['HORIZON-ERC-2021-VICECHAIRS-IBA' 'ERC-2025-NCPS-IBA'
 'HORIZON-ERC-2021-arXiv-IBA'
 'HORIZON-CL4-2024-SSA-SST-SD-IBA-to-be-deleted'
 'HORIZON-CL4-2024-SSA-SST-AE-IBA-to-be-deleted'
 'HORIZON-ERC-2023-VICECHAIRS-IBA' 'HORIZON-ERC-2022-VICECHAIRS-IBA'
 'HORIZON-CL4-2024-SSA-SST-SP-IBA-to-be-deleted']
MSCA : destination_code à null après traitement
['HORIZON-MSCA-2021-INCO-01-01' 'HORIZON-MSCA-2021-FRC-IBA'
 'HORIZON-MSCA-2022-ALUMNI-IBA' 'HORIZON-MSCA-2024-NCP-01-01'
 'HORIZON-MSCA-2023-FTP-01-01' 'HORIZON-MSCA-2022-Ukraine-ART195-IBA'
 'HORIZON-MSCA-2021-RR-01-01' 'HORIZON-MSCA-2024-RR-01-01'
 'HORIZON-MSCA-2023-BEL-IBA' 'HORIZON-MSCA-2024-UKRAINE-IBA'
 'HORIZON-MSCA-2024-INCO-01-01' 'HORIZON-MSCA-SNLS-2021-IBA'
 'HORIZON-MSCA-2021-NCP-01-01' 'HORIZON-MSCA-2024-ALUMNI-IB

c:\Users\zfriant\Documents\GitHub\pcri\step1_mainData\topics.py:171: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SESAR' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top.loc[mask, 'destination_code'] = i


size merged after add topics: 102295

### ACTIONS
1- call sans action:
                                      call_id
24693  HORIZON-HLTH-2022-DISEASE-06-two-stage

- size merged after add actions: 102295


### CALLS

### CALLS+MERGED
2 - CALL_ID de merged -> nb call+deadline: 530, nb call unique: 492 

### CALLS to CHECK
- CALLS de base calls with merge call_id ->
nb call+deadline: 537, nb call unique: 492 
2- si calls pas dans call_id :
                             call_id call_deadline
231  HORIZON-EIC-2023-BOOSTER-IBA-01    2023-07-05
248  HORIZON-EIC-2024-ACCELERATOR-02    2024-10-03
251  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-09-04
252  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-11-14
467       HORIZON-MSCA-2021-SNLS-IBA    2021-07-27
505    HORIZON-WIDERA-2022-ACCESS-07    2022-04-20
522    HORIZON-WIDERA-2023-ACCESS-06    2024-09-26
3- calls with multi deadline
                                     call_id call_deadline   call_budget
17                              ERC-2023-POC    

In [83]:
# def entities_single_create(df, lien):

#     print("\n### ENTITIES SINGLE")
#     entities_single=df.groupby(['generalPic', 'country_code_mapping']).head(1)
#     print(f"- size entities after one selection pic+cc: {len(entities_single)}")

#     print(f"\n- {entities_single.generalState.value_counts()}")
#     if (entities_single.generalPic.nunique())==(lien.generalPic.nunique()):
#         print(f"\n1 - nombre de pics OK")
#     #si pas le m^me nombre de pics entre lien et entities
#     elif len(set(lien.generalPic.unique()))>len(set(df.generalPic.unique())):
#         pic_lien=list(set(lien.generalPic.unique()) - set(df.generalPic.unique()))
#         print(f"\n2 - pic_lien absent de entities_single {pic_lien}; faire code")

#     tmp=entities_single.groupby(['generalPic', 'country_code_mapping']).filter(lambda x: x['generalPic'].count() > 1.)
#     if not tmp.empty:
#         print(f"1 - ATTENTION doublon generalPic revoir code ci-dessous si besoin")
           
#     print(f"- size entities_single:{len(entities_single)}")
#     return entities_single


# entities_single = entities_single_create(entities, lien)
entities_single = entities_single_create(entities, lien)

def entities_info_create(entities_single, lien):
    print("\n### ENTITIES INFO")
    entities_info = (entities_single
                     .drop(['pic', 'cedex', 'countryCode_y','lastUpdateDate'], axis=1)
                     .drop_duplicates())

    if len(entities_info[['generalPic', 'country_code_mapping']].drop_duplicates())!=len(lien[['generalPic', 'country_code_mapping']].drop_duplicates()):
        print(f"1- ATTENTION ! size genPic+cc -> entities_info : \n{len(entities_info[['generalPic', 'country_code_mapping']].drop_duplicates())},  lien:{len(lien[['generalPic', 'country_code_mapping']].drop_duplicates())}")
    else:
        pass
    print(f"- size entities_info: {len(entities_info)}")
    return entities_info
entities_info = entities_info_create(entities_single, lien)
entities_info.columns


### ENTITIES SINGLE
- size entities after one selection pic+cc: 77984

- generalState
VALIDATED     40339
DECLARED      35815
SLEEPING       1524
DEPRECATED      277
SUSPENDED        17
BLOCKED          12
Name: count, dtype: int64

1 - nombre de pics OK
- size entities_single:77984

### ENTITIES INFO
1- ATTENTION ! size genPic+cc -> entities_info : 
77984,  lien:77967
- size entities_info: 77984


Index(['generalPic', 'countryCode', 'legalName', 'businessName',
       'generalState', 'street', 'postalCode', 'postalBox', 'city', 'nutsCode',
       'webPage', 'isSme', 'isNonProfit', 'isPublicBody',
       'isInternationalOrganisation', 'isResearchOrganisation',
       'isHigherEducation', 'legalEntityTypeCode', 'legalEntityType',
       'legalType', 'legalStatus', 'vat', 'legalRegNumber', 'naceCode',
       'gps_loc', 'country_code_mapping'],
      dtype='object')

In [84]:
# ref_source = ref_source_load('ref')
# ref, genPic_to_new = ref_source_2d_select(ref_source, 'HE')

def entities_tmp_create(entities_info, countries, ref):
    print("### create ENTITIES TMP pour ref")
    tab = entities_info.merge(countries[['country_code_mapping', 'country_name_mapping', 'countryCode_parent']], how='left', on='country_code_mapping')
    tmp = tab.merge(ref, how='inner', on=['generalPic','country_code_mapping'])
    print(f"- size entities_info before:{len(entities_info)}, size entities_info+ref -> tmp:{len(tmp)}, Pic unique tmp:{len(tmp.generalPic.unique())}")
    # entities only into entities_info
    print("# missing entities into ref")
    tmp2 = tab.merge(tmp[['generalPic','country_code_mapping']], how='left', on=['generalPic','country_code_mapping'], indicator=True).query('_merge=="left_only"').drop(columns=['_merge'])
    print(f"- entities_info en + -> (tmp2): {len(tmp2)}")
    if not tmp2.empty:
        # test lien avec ref voire si un identifiant seulement sur le generalPic
        tmp2 = tmp2.merge(ref.drop(columns='country_code_mapping').drop_duplicates(), how='inner', on='generalPic')
        print(f"- size lien tmp2 with ref: {len(tmp2)}")
        ## add tmp2 to tmp
        tmp1 = pd.concat([tmp, tmp2], ignore_index=True)
        # entities_info without id
        tmp1 = tab.merge(tmp1[['generalPic','country_code_mapping']], how='left',on=['generalPic','country_code_mapping'], indicator=True).query('_merge=="left_only"').drop(columns=['_merge'])
        print(f"- size entities_info without id -> tmp1: {len(tmp1)}")
        tmp = pd.concat([tmp1, tmp], ignore_index=True)

    if (len(tmp))!=(len(entities_info)):
        print(f"1 - ATTENTION!!! size result {len(tmp)} diff size entities_info {len(entities_info)}")
    print(f"- End size entities_tmp {len(tmp)}")
    return tmp
entities_tmp = entities_tmp_create(entities_info, countries, ref)
print(f"size entities_tmp: {len(entities_tmp)}")
entities_tmp = entities_for_merge(entities_tmp)

### create ENTITIES TMP pour ref
- size entities_info before:77984, size entities_info+ref -> tmp:77963, Pic unique tmp:77898
# missing entities into ref
- entities_info en + -> (tmp2): 21
- size lien tmp2 with ref: 0
- size entities_info without id -> tmp1: 21
- End size entities_tmp 77984
size entities_tmp: 77984
1 - After add ref to entities: 77984

Index(['generalPic', 'legalName', 'businessName', 'id', 'id_secondaire',
       'ZONAGE', 'country_code_mapping', 'countryCode_parent'],
      dtype='object')
2 - size entities si multi id -> entities_size_to_keep = 77988


In [85]:
ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
def merge_ror(entities_tmp, ror, countries):
    print("### merge ROR")
    ccode=json.load(open("data_files/countryCode_match.json"))
    for i in ccode:
        for k,v in i.items():
            ror.loc[ror.country_code==k, 'country_code'] = v
            ror.loc[ror.country_code==k, 'country_code'] = v
    ror = (ror
           .merge(countries[['countryCode', 'country_code_mapping']], 
                  how='left', left_on='country_code', right_on='countryCode')
            .drop(columns=['countryCode', 'country_code']))

    entities_tmp = (entities_tmp
                    .merge(ror.drop(columns='country_code_mapping'), 
                           how='left', left_on=['id_extend'], right_on=['id_source'])
                    .drop(columns='id_source')
                    .drop_duplicates())
    print(f"- End size entities_tmp+ror_info: {len(entities_tmp)}")
    if any(entities_tmp.groupby('generalPic')['generalPic'].transform('count')>1):
        entities_tmp[entities_tmp.groupby('generalPic')['generalPic'].transform('count')>1]
    return entities_tmp
ror
entities_tmp = merge_ror(entities_tmp, ror, countries)

### merge ROR
- End size entities_tmp+ror_info: 77988


In [86]:
paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
# paysage_category = IDpaysage_category(paysage)
paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
cat_filter = category_paysage(paysage_category)
entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)
sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
entities_tmp = merge_sirene(entities_tmp, sirene)

entities_tmp.loc[(~entities_tmp.id.isnull())&(entities_tmp.entities_id.isnull()), 'entities_id'] = entities_tmp.id

if any(entities_tmp.siren.str.contains(';', na=False)):
    print("ATTENTION faire code pour traiter deux siren différents -> ce qui serait bizarre qu'il y ait 2 siren")


### CATEGORY paysage
### merge PAYSAGE
- doublons PIC
      generalPic                                          legalName  \
119    876941596                       WEIR MINERALS EUROPE LIMITED   
120    876941596                       WEIR MINERALS EUROPE LIMITED   
173    877256167                                  EURACTIV MEDIA BV   
174    877256167                             EURACTIV MEDIA NETWORK   
320    877625834  NEPTUNE LINES SHIPPING AND MANAGING ENTERPRISE...   
321    877625834  NEPTUNE LINES SHIPPING AND MANAGING ENTERPRISE...   
1738   878469540                                 MATTHIAS DAVID HUW   
1739   878469540                                 MATTHIAS DAVID HUW   
3009   879068903                            ROSING JAKOBSEN PAARNAQ   
3010   879068903                            ROSING JAKOBSEN PAARNAQ   
3202   879162896                                     TORRASPAPEL SA   
3203   879162896                                     TORRASPAPEL SA   
4534   879924055  INSTI

In [87]:
entities_tmp = IDpic(entities_tmp)
entities_tmp = entities_tmp.merge(get_source_ID(entities_tmp, 'entities_id'), how='left', on='entities_id')

### groupe entreprises
# groupe = groupe_treatment('groupe_prov', 'groupe')
### si besoin de charger groupe 
groupe = pd.read_pickle(f"{PATH_REF}groupe.pkl")
print(f"taille de entities_tmp avant groupe:{len(entities_tmp)}")
entities_tmp = merge_groupe(entities_tmp, groupe)



### traitement id_pic avec tiret
- End size entities_tmp: 77988
### sourcer les identifiants pour getInformations
taille de entities_tmp avant groupe:77988

### merge avec GROUPE
taille de entities_tmp après groupe 77988


In [88]:
entities_tmp = entities_clean(entities_tmp)
entities_check_null(entities_tmp)

# traitement catégorie
entities_tmp = category_woven(entities_tmp, sirene)
entities_tmp = category_agreg(entities_tmp)
def entities_info_add(entities_tmp, entities_info, countries):
    print("\n### entities_info + entities_tmp")
        #ajout des infos country à participants_info
    entities_info = (entities_info
                    .merge(countries[['country_code_mapping', 'country_name_mapping', 'country_code']], how='left', on='country_code_mapping'))
        
    entities_info = (entities_info
        .merge(entities_tmp[
            ['generalPic', 'id', 'ZONAGE', 'id_m', 'siren', 'country_code_mapping',
            'id_secondaire', 'entities_id',  'entities_name', 'entities_acronym', 
            'insee_cat_code', 'insee_cat_name',  'category_agregation',
            'paysage_category', 'flag_entreprise', 
            'ror_category', 'category_woven', 'source_id', 'sector',  
            'siret_closeDate', 'cat_an',
            'groupe_name','groupe_acronym', 'groupe_id', 'groupe_sector']],
        how='left', on=['generalPic', 'country_code_mapping'])
        .drop(columns=['legalName', 'businessName']))
    print(f"- size entities_info + entities_tmp: {len(entities_info)}")
    return entities_info
entities_info = entities_info_add(entities_tmp, entities_info, countries)
entities_info = cordis_type(entities_info)

entities_info = fix_countries(entities_info, countries)
entities_info = mires(entities_info)

### ENTITIES TMP cleaning name
- size entities_tmp without entities_name: 59272
Series([], Name: legalName, dtype: object)
- End size entities_tmp without entities_name: 59271

## check entities null

## category woven
- categorization missing
Empty DataFrame
Columns: [source_id, entities_name, entities_id, siren_cj, paysage_category]
Index: []
- taille de df après cat: 77987

### entities_info + entities_tmp
- size entities_info + entities_tmp: 77988
### CORDIS type
- size entities_info: 77988

### entities_info + countries
- longueur entities_info après ajout calculated_country : 77988
Index(['generalPic', 'countryCode', 'generalState', 'street', 'postalCode',
       'postalBox', 'city', 'nutsCode', 'webPage', 'cordis_is_sme',
       'isNonProfit', 'isPublicBody', 'isInternationalOrganisation',
       'isResearchOrganisation', 'isHigherEducation', 'legalType', 'vat',
       'legalRegNumber', 'naceCode', 'gps_loc', 'country_code_mapping',
       'country_name_mapping', 'country_code',

In [89]:
print("### check enties fr avec id commençant par pic")
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(entities_info[(entities_info.country_code=='FRA')&(entities_info.entities_id.str.contains('pic'))][['entities_id', 'entities_name']])


### check enties fr avec id commençant par pic
        entities_id                                      entities_name
1222   pic878172138                                    Georges cahuzac
2431   pic878833969                                        Chp consult
2717   pic878950563  Conseil régional de l'ordre des experts-compta...
3112   pic879123320                             Astremiha therapeutics
3265   pic879213821                           Bristol-myers squibb sas
3309   pic879238847                                         Le bourdon
3488   pic879350882                                       Finis terrae
3753   pic879493278                                      Bao formation
4309   pic879868280                                             Avicom
4422   pic879903685                             Musée nicéphore niépce
5361   pic880310115                                     Denis lombardi
5536   pic880413614  Association française pour l'education par la ...
5637   pic880483260           

In [90]:

file_name = f"{PATH_CLEAN}entities_info_current2.pkl"
with open(file_name, 'wb') as file:
    pd.to_pickle(entities_info, file)

In [91]:
def entities_with_lien(entities_info, lien, genPic_to_new):
    print("### LIEN + entities_info -> pour calculations")
    print(f"- ETAT avant lien ->\ngeneralPic de lien={lien.generalPic.nunique()},\ngeneralPic de entities_info={entities_info.generalPic.nunique()}")

    lien = lien.merge(genPic_to_new, how='left', on=['generalPic', 'country_code_mapping'])
    lien = lien.rename(columns={'generalPic':'pic_old', 'pic_new':'generalPic'})
    lien.loc[lien.generalPic.isnull(), 'generalPic'] = lien.loc[lien.generalPic.isnull(), 'pic_old']

    part_step = (lien
                .merge(entities_info[
                    ['generalPic', 'flag_entreprise', 'groupe_id', 'groupe_name', 'groupe_acronym',
                    'cordis_is_sme', 'cordis_type_entity_code', 'cordis_type_entity_name_fr', 
                    'cordis_type_entity_name_en', 'cordis_type_entity_acro', 'nutsCode',
                    'country_code', 'country_code_mapping', 'extra_joint_organization']].drop_duplicates(),
                    how='left', on=['generalPic', 'country_code_mapping'])
                .drop(columns={'n_app', 'n_part', 'participant_pic'})
                .rename(columns={ 'nutsCode':'entities_nuts', 'nuts_code':'participation_nuts'})   
            )

    if len(part_step)==len(lien):
        print(f'1- part_step ({len(part_step)}) = lien')
    else:
        print(f"2- lien={len(lien)}, part_step={len(part_step)}")
    return part_step

part_step = entities_with_lien(entities_info, lien, genPic_to_new)

### LIEN + entities_info -> pour calculations
- ETAT avant lien ->
generalPic de lien=77919,
generalPic de entities_info=77919
2- lien=470876, part_step=470877


In [94]:
proj_no_coord = proj_no_coord(projects)
part_prop = applicants_calcul(part_step, app1)
part_proj = participants_calcul(part_step, part)
participation = participations_complete(part_prop, part_proj, proj_no_coord)
del part_proj, part_prop



### CALCULS applicants
0 - 208,807,113,973.0, 208,782,564,667.7
1- ATTENTION ! -73 participations perdues entre app1 et subv_p
3- ATTENTION ! Ecart subventions proposals -> subv_orig:208,807,113,973.0, après fusion:208,782,564,667.7
5- ATTENTION ! bien vérifier le volume de calculated_proposal_subv dans PARTICIPATION FINALE :208,782,564,667.7, subv_orig:208,807,113,973.0
- size part_prop: 463463

### CALCULS participants
1- ATTENTION ! participations perdues entre 100423 de part1 et 100417 de subv_pt
3- Etape part_step/part1 -> beneficiary_subv OK
5- Etape part_step/part1 -> calculated_fund OK
- size part_prop: 100428
### PARTICIPATIONS final
- control role: ['coordinator' 'partner']
- size participation: 563891
